In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
n_input = 784
n_classes = 10
learning_rate = 0.01
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [3]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([5,5, 1, 32])),
    'wc2' : tf.Variable(tf.random_normal([5,5, 32, 64])),
    'whl' : tf.Variable(tf.random_normal([7 *7 * 64, 1024])),
    'out' : tf.Variable(tf.random_normal([1024, n_classes])),
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bhl' : tf.Variable(tf.random_normal([1024])),
    'outB' : tf.Variable(tf.random_normal([n_classes])),
}

In [4]:
def conv(x, w, b, strides=1):
    x = tf.nn.conv2d(x, w, padding='SAME', strides=[1,strides, strides,1])
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding='SAME', ksize=[1, k, k, 1], strides=[1,k, k, 1])
    
def cnn_result(x, weights, biases) :
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    conv1 = conv(x, weights['wc1'], biases['bc1'])
    conv1 = maxpooling(conv1, k = 2)
    
    conv2 = conv(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpooling(conv2, k = 2)
    
    hidden_input = tf.reshape(conv2, [-1, 7 * 7 * 64])
    hidden_output_before_relu = tf.add(tf.matmul(hidden_input, weights['whl']), biases['bhl'])
    hidden_output = tf.nn.relu(hidden_output_before_relu)
    
    out = tf.add(tf.matmul(hidden_output, weights['out']), biases['outB'])
    return out

In [ ]:
pred = cnn_result(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()
batch_size = 100
sess = tf.Session()
sess.run(init)
for i in range(25):
    num_batches = 550
    sum_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run([optimizer], feed_dict={x: batch_x, y : batch_y})